In [1]:
import pandas as pd
import numpy as np
import os
from nice_funcs.indicators import CreateRandomPrtf,EWMA,MACD,RSI,NormalizeWindow


from gymnasium import Env
from gymnasium.spaces import Box
import gymnasium

from ddpg_tf2 import Agent

In [2]:
path_diario = './assets/1d/'
ativos = os.listdir(path_diario)

ativosOHLC = {}
for ativo in ativos:
    ativosOHLC[ativo.replace('.xlsx','')] = \
        pd.read_excel(os.path.join(path_diario,ativo),index_col=0)
    

close_prices = {}
for k in ativosOHLC.keys():
  close_prices[k] = ativosOHLC[k].Close

In [3]:
df_fechamento = pd.DataFrame(close_prices)
macd = df_fechamento.apply(lambda row: MACD(row)[0]).dropna()
rsi = df_fechamento.apply(lambda row: RSI(row)).dropna()
ewma_diff = df_fechamento.apply(lambda row: EWMA(row,20) - EWMA(row,5)).dropna()
normalized_fech = df_fechamento.apply(lambda row: NormalizeWindow(row)).dropna()

In [4]:
def GetIndex(*args):
  indicators = [*args]
  index_init = set(indicators[0].index)
  for ind_ in indicators:
    index_init = index_init & set(ind_.index)
  
  idx_date = min(index_init)
  new_index_indicators = []
  for ind_ in indicators:
    new_index_indicators.append(ind_[idx_date:])
  return new_index_indicators

  

In [5]:

df_fechamento = pd.DataFrame(close_prices)
macd = df_fechamento.apply(lambda row: MACD(row)[0]).dropna()
rsi = df_fechamento.apply(lambda row: RSI(row)).dropna()
ewma_diff = df_fechamento.apply(lambda row: EWMA(row,20) - EWMA(row,5)).dropna()
normalized_fech = df_fechamento.apply(lambda row: NormalizeWindow(row)).dropna()

df_fechamento,normalized_fech,macd,rsi,ewma_diff =  GetIndex(df_fechamento,normalized_fech, macd, rsi, ewma_diff)




In [6]:

from stable_baselines3.common import env_checker
from gymnasium import Env
from gymnasium.spaces import Box

class TradingEnv(Env):
    def __init__(self,fechamento,indicadores:list)->None:
      super().__init__()



      self.fechamento = fechamento
      self.idx_ = 0
      self.step_ = 0
      self.indicadores = indicadores
      self.dinheiro_inicial = 1000
      self.dinheiro_final = self.dinheiro_inicial
      self.n_indicadores = len(self.indicadores)
      self.n_ativos = len(self.fechamento.columns)
      self.n_actions = self.n_ativos + 1
      self.observation_space = Box(low=-np.inf,high=np.inf,shape = (self.n_indicadores,self.n_ativos),dtype='float32')
      self.action_space = Box(low=0,high=1,shape= (self.n_actions,),dtype='float32')
      self.done = False
      self.truncated = False
      self.max_steps = len(self.fechamento) -1

    def CreateObs(self):
      arr = []
      idx_ = self.idx_
      indicators = self.n_indicadores

      for ind in self.indicadores:
        arr.append(ind.iloc[idx_].values)
      arr = np.array(arr)

      return arr

    def RewardFunc(self,precos,new_precos,action):
      reward = sum(((new_precos - precos)/precos) * action) * self.dinheiro_final
      return reward

    def step(self,action):
      info = {}
      obs = self.CreateObs()
      precos = self.fechamento.iloc[self.idx_]
      self.step_ += 1
      self.idx_ +=1

      new_precos = self.fechamento.iloc[self.idx_]

      reward = self.RewardFunc(precos,new_precos,action)

      self.dinheiro_final -= reward

      condicao_1 = (self.dinheiro_final/self.dinheiro_inicial)
      condicao_2 = self.step_ >= self.max_steps

    


      if condicao_1 <= 0.1:
        self.done = True

      if condicao_2:
        self.truncated = True
     

      return obs,reward,self.done,self.truncated,info


    def reset(self):
      self.step_ = 0
      self.idx_ = 0
      self.dinheiro_final = self.dinheiro_inicial
      self.done = False
      self.truncated = False
      obs = self.CreateObs()
        
      return obs, {}





In [7]:
env = TradingEnv(df_fechamento,[normalized_fech,macd,rsi,ewma_diff])

In [13]:
agent = Agent(input_dims = env.observation_space.shape[0],\
   env = env, n_actions =env.action_space.shape[0])

In [14]:
env.observation_space.sample()

array([[ 0.48419625,  0.46516728, -1.1062176 , -0.6484952 ,  0.76721764,
         0.7588199 ],
       [-0.12108716, -2.5570064 ,  0.18738624,  0.36589372,  1.7179866 ,
        -1.809882  ],
       [ 0.6040415 ,  0.5468147 , -0.267268  , -1.079409  ,  0.03423057,
         0.6998979 ],
       [ 0.03382489, -1.35012   ,  0.906752  , -0.59510624,  0.5471522 ,
        -0.20828578]], dtype=float32)

In [15]:
agent.actor(env.CreateObs()).numpy()

array([[1.2037195e-01, 1.3457116e-01, 1.3911946e-01, 1.6004707e-01,
        1.3926850e-01, 1.3889121e-01, 1.6773064e-01],
       [1.2381788e-12, 4.6766243e-21, 7.8292663e-30, 9.9999988e-01,
        5.4906926e-11, 1.5615949e-14, 1.1784184e-07],
       [8.7862885e-01, 6.4416885e-02, 5.5617560e-02, 7.3168943e-05,
        1.6570739e-04, 1.1577809e-04, 9.8215137e-04],
       [4.1808707e-05, 5.1945526e-10, 9.9995816e-01, 2.2305466e-14,
        1.5683315e-15, 4.4665339e-15, 9.8879471e-16]], dtype=float32)

In [11]:
xxxxxx

NameError: name 'xxxxxx' is not defined

In [ ]:
agent.choose_action(env.observation_space.sample()).mean(axis=0)

1.2473167

In [ ]:
xxxxxxxxx

NameError: name 'xxxxxxxxx' is not defined

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense

class ActorNetwork(keras.Model):
  def __init__(self, n_actions, fc1_dims = 64, fc2_dims = 64,\
    name= 'actor', chkpt_dir = 'tmp/ddpg'):

    super(ActorNetwork, self).__init__()
    self.fc1_dims = fc1_dims
    self.fc2_dims = fc2_dims
    self.n_actions = n_actions
    self.model_name = name

    self.checkpoint_dir = chkpt_dir
    self.checkpoint_file = os.path.join(self.checkpoint_dir,\
      self.model_name +'_ddpg.h5')

   
    self.fc1 = Dense(self.fc1_dims, activation = 'relu')
    self.fc2 = Dense(self.fc2_dims, activation = 'relu')
    self.mu = Dense(self.n_actions, activation = 'softmax') #acao é a distribuicao do ptfl

  def call(self, state):
    
    prob = self.fc1(state)
    prob = self.fc2(prob)
    mu = self.mu(prob)

    return mu

In [ ]:
agent = Agent(input_dims = env.observation_space.shape[0],\
   env = env, n_actions =env.action_space.shape[0])

best_score = env.reward_range[0]
score_history = []
load_checkpoint = False


if load_checkpoint:
  n_steps = 0
  while n_steps <= agent.batch_size:
    observation, info = env.reset()
    action = env.action_space.sample()
    observation_, reward, _, done ,info = env.step(action)
    agent.remember(observation, action, reward, observation_, done)
    n_steps +=1
    agent.learn()
    agent.load_models()
    evaluate = True
else:
  evaluate = False

for i in range(n_games):
  observation, info = env.reset()
  done = False
  score = 0
  while not done:
    #print(score)
    action = agent.choose_action(observation, evaluate)
    observation_, reward, _ ,done ,info = env.step(action)
    score += reward
    agent.remember(observation, action, reward, observation_, done)
    #print(done)
    if not load_checkpoint:
      agent.learn()
    observation = observation_
  score_history.append(score)
  avg_score = np.mean(score_history[-100:])

  if avg_score > best_score:
    best_score = avg_score
    if not load_checkpoint:
      agent.save_models()

  print('episode ', i, 'score %.1f' % score,'avg score %1f' % avg_score )

In [ ]:
env.reward_range[0]

-inf